In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/HI_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14
1,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14
2,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14
3,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14
4,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14
1,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14
2,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14
3,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14
4,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
680,4509797009,HI,"99-1405A KOAHA PL, AIEA, HI",21.372393,-157.902690,NaN
1240,9138068502,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,NaN
1983,5751787107,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,NaN
3407,1221157100,HI,"803 KAMEHAMEHA HWY STE 416, PEARL CITY, HI",21.392535,-157.967690,NaN
4512,5148568309,HI,"1630 Farrington Ave., Kualapuu, HI",21.152936,-157.036243,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('15')] #HI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/5 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

5


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14796/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
680,4509797009,HI,"99-1405A KOAHA PL, AIEA, HI",21.372393,-157.902690,[150030075072000]
1240,9138068502,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,[150090304063001]
1983,5751787107,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,[150090304063001]
3407,1221157100,HI,"803 KAMEHAMEHA HWY STE 416, PEARL CITY, HI",21.392535,-157.967690,[150030080011002]
4512,5148568309,HI,"1630 Farrington Ave., Kualapuu, HI",21.152936,-157.036243,[150090318012001]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14796/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
680,4509797009,HI,"99-1405A KOAHA PL, AIEA, HI",21.372393,-157.902690,150030075072000
1240,9138068502,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,150090304063001
1983,5751787107,HI,"2706 KAMELANI LOOP, MAKAWAO, HI",20.841369,-156.353242,150090304063001
3407,1221157100,HI,"803 KAMEHAMEHA HWY STE 416, PEARL CITY, HI",21.392535,-157.967690,150030080011002
4512,5148568309,HI,"1630 Farrington Ave., Kualapuu, HI",21.152936,-157.036243,150090318012001


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
680,4509797009,150030075072000
1240,9138068502,150090304063001
1983,5751787107,150090304063001
3407,1221157100,150030080011002
4512,5148568309,150090318012001


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/HI_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,290450,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14
1,290451,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14
2,290452,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14
3,290453,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14
4,290454,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,290450,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14,NaN
1,290451,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14,NaN
2,290452,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14,NaN
3,290453,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14,NaN
4,290454,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14,NaN
...,...,...,...,...,...,...,...,...
4974,295424,9153468404,HI,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,1.500903e+14,NaN
4975,295425,9206638401,HI,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,1.500301e+14,NaN
4976,295426,9742598602,HI,"99 Makaala St, Hilo, HI",19.702781,-155.067456,1.500102e+14,NaN
4977,295427,9937897210,HI,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,1.500301e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,290450,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,1.500300e+14,NaN,150030025001006.0
1,290451,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,1.500300e+14,NaN,150030040001011.0
2,290452,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,1.500300e+14,NaN,150030020031001.0
3,290453,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,1.500300e+14,NaN,150030018012001.0
4,290454,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,1.500300e+14,NaN,150030005005001.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4424627005,HI,"1765 SOUTH KING ST, HONOLULU, HI",21.295594,-157.833156,150030025001006.0
1,5613647004,HI,"841 BISHOP ST ST 1110, HONOLULU, HI",21.307333,-157.861915,150030040001011.0
2,4281497103,HI,"307 Lewers St. #210 0.0, Honolulu, HI",21.280410,-157.828942,150030020031001.0
3,5918468104,HI,"2552 KALAKAUA AVENUE, HONOLULU, HI",21.273206,-157.822375,150030018012001.0
4,5683347210,HI,"5000 Kahala Ave, Honolulu, HI",21.272182,-157.772822,150030005005001.0
...,...,...,...,...,...,...
4974,9153468404,HI,"15 S Wakea Ave Temptation Tours Inc, Kahului, HI",20.887782,-156.477595,150090319001108.0
4975,9206638401,HI,"91-1600 Wahane St Awiwohi Bros Inc, Kapolei, HI",21.349410,-158.067337,150030086231001.0
4976,9742598602,HI,"99 Makaala St, Hilo, HI",19.702781,-155.067456,150010206004015.0
4977,9937897210,HI,"1454 UILA ST., HONOLULU, HI",21.365520,-157.922392,150030075084001.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z


Convert to File

In [19]:
fips_HI = fips_merge_drop

In [20]:
fips_HI.to_csv("../../../data/state_data/geo/geo_fips/23/HI_fips_scraped.csv") 